In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
raw_data = pd.read_csv('./dataset.csv', sep=',')
for index in range(len(raw_data)):
    if raw_data.at[index, 'star_rating'] > 3:
        raw_data.at[index, 'star_rating'] = 1
    elif raw_data.at[index, 'star_rating'] == 3:
        raw_data.at[index, 'star_rating'] = 2
    else:
        raw_data.at[index, 'star_rating'] = 0
from bs4 import BeautifulSoup
import re
# data cleaning
raw_data['review_body'] = raw_data['review_body'].str.lower()
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: re.sub(r'http\S+', '', BeautifulSoup(x, 'html.parser').get_text()))
def remove_non_alphabetical(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

raw_data['review_body'] = raw_data['review_body'].apply(remove_non_alphabetical)
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: ' '.join(x.split()))
contraction_dict = contraction_dict = {
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "you're": "you are",
    "you've": "you have",
    "you'll": "you will",
    "when's": "when is",
    "let's": "let us",
    "'cause": "because",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "couldn't": "could not",
    "wasn't": "was not",
    "weren't": "were not",
    "I'm": "I am",
    "I've": "I have",
    "I'll": "I will",
    "it's": "it is",
    "that's": "that is",
    "who's": "who is",
    "what's": "what is",
    "where's": "where is",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "she's": "she is",
    "he's": "he is",
    "how's": "how is",
    "you'd": "you would",
    "we'd": "we would",
    "they'd": "they would",
}
raw_data['review_body'] = raw_data['review_body'].replace(contraction_dict, regex=True)
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    words = text.split(" ")
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
raw_data['review_body'] = raw_data['review_body'].apply(remove_stop_words)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4') # I have to download this dataset
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import string
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english') + list(string.punctuation))
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens
raw_data['clean_reviw'] = raw_data['review_body'].apply(preprocess_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  app.launch_new_instance()
C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:17: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  app.launch_new_instance()
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...


In [2]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('word2vec_model_b.txt', binary=False)

In [3]:
# binary dataset
values_to_stratify = [0,1]
def stratified_sample(group, n=100000):
    return group.sample(n=n, random_state=1)

binary_data= raw_data[raw_data['star_rating'].isin(values_to_stratify)].groupby('star_rating', group_keys=False).apply(stratified_sample)

def average_word2vec(text, model=None, vw=None):
    if model:
        word_vectors = [model[word] for word in text if word in model.key_to_index]
    if vw:
        word_vectors = [wv[word] for word in text if word in wv.key_to_index]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(300)
binary_data['word2vec_features_self'] = binary_data['clean_reviw'].apply(lambda x: average_word2vec(x, model=model))

In [4]:
from sklearn.utils import shuffle
shuffled_dataset = shuffle(binary_data, random_state=42)
from sklearn.model_selection import train_test_split
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(shuffled_dataset['word2vec_features_self'].tolist(), shuffled_dataset['star_rating'], test_size=0.2, random_state=42)


# Binary

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

X_train_tensor_b_s = torch.FloatTensor(X_train_s)
y_train_tensor_b_s = torch.LongTensor(y_train_s.values)
X_test_tensor_b_s = torch.FloatTensor(X_test_s)
y_test_tensor_b_s = torch.LongTensor(y_test_s.values)

input_size = 300
hidden_size1 = 50
hidden_size2 = 10
output_size_binary = 2
output_size_ternary = 3

# model
class MyMLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MyMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

mlp_binary = MyMLP(input_size, hidden_size1, hidden_size2, output_size_binary)
criterion = nn.CrossEntropyLoss()
optimizer_binary = optim.Adam(mlp_binary.parameters(), lr=0.001)
for epoch in range(1500):
    outputs = mlp_binary(X_train_tensor_b_s)
    loss = criterion(outputs, y_train_tensor_b_s)
    optimizer_binary.zero_grad()
    loss.backward()
    optimizer_binary.step()
    
with torch.no_grad():
    mlp_binary.eval()
    y_pred_binary_s = torch.argmax(mlp_binary(X_test_tensor_b_s), dim=1)
    accuracy_binary_s = torch.sum(y_pred_binary_s == y_test_tensor_b_s).item() / len(y_test_tensor_b_s)
    
print(accuracy_binary_s)

C:\Users\monkeydc\.conda\envs\561\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  if sys.path[0] == "":


0.8617


# Ternary

In [6]:
def average_word2vec(text, model=None, vw=None):
    if model:
        word_vectors = [model[word] for word in text if word in model.key_to_index]
    if vw:
        word_vectors = [wv[word] for word in text if word in wv.key_to_index]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(300)
    
raw_data['word2vec_features_self'] = raw_data['clean_reviw'].apply(lambda x: average_word2vec(x, model=model))

from sklearn.utils import shuffle
shuffled_dataset = shuffle(raw_data, random_state=42)
from sklearn.model_selection import train_test_split
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(shuffled_dataset['word2vec_features_self'].tolist(), shuffled_dataset['star_rating'], test_size=0.2, random_state=42)

X_train_tensor_t_g = torch.FloatTensor(X_train_g)
y_train_tensor_t_g = torch.LongTensor(y_train_g.values)
X_test_tensor_t_g = torch.FloatTensor(X_test_g)
y_test_tensor_t_g = torch.LongTensor(y_test_g.values)

mlp_ternary = MyMLP(input_size, hidden_size1, hidden_size2, output_size_ternary)
criterion = nn.CrossEntropyLoss()
optimizer_ternary = optim.Adam(mlp_ternary.parameters(), lr=0.001)
for epoch in range(1500):
    outputs = mlp_ternary(X_train_tensor_t_g)
    loss = criterion(outputs, y_train_tensor_t_g)
    optimizer_ternary.zero_grad()
    loss.backward()
    optimizer_ternary.step()
with torch.no_grad():
    mlp_ternary.eval()
    y_pred_ternary_g = torch.argmax(mlp_ternary(X_test_tensor_t_g), dim=1)
    accuracy_ternary_g = torch.sum(y_pred_ternary_g == y_test_tensor_t_g).item() / len(y_test_tensor_t_g)

print(accuracy_ternary_g)

0.70174
